In [134]:
# https://preppindata.blogspot.com/2021/03/2021-week-11-cocktail-profit-margins.html

import pandas as pd
import numpy as np

### Input the dataset 

In [135]:
df_cocktails = pd.read_excel(r'C:\Users\StanleyChan\SynologyDrive\Work Backup\my document\python\Preppin Data\data\PD 2021 Wk 11 Input.xlsx',sheet_name='Cocktails')
df_sourcing = pd.read_excel(r'C:\Users\StanleyChan\SynologyDrive\Work Backup\my document\python\Preppin Data\data\PD 2021 Wk 11 Input.xlsx',sheet_name='Sourcing')
df_rates = pd.read_excel(r'C:\Users\StanleyChan\SynologyDrive\Work Backup\my document\python\Preppin Data\data\PD 2021 Wk 11 Input.xlsx',sheet_name='Conversion Rates')


### Split out the recipes into the different ingredients and their measurements

In [136]:
df_cocktails['Recipe (ml)'] = df_cocktails['Recipe (ml)'].str.split('; ')
df_cocktails = df_cocktails.explode('Recipe (ml)')

df_cocktails[['Recipe','ml']] = df_cocktails['Recipe (ml)'].str.extract('(.*):(\d+)ml',expand=True)
df_cocktails

,Cocktail,Price (£),Recipe (ml),Recipe,ml
0,Raspberry Lemon Drop,8.50,Citroen Vodka:45ml,Citroen Vodka,45
0,Raspberry Lemon Drop,8.50,Chambord:20ml,Chambord,20
0,Raspberry Lemon Drop,8.50,Triple Sec:20ml,Triple Sec,20
1,Bay Breeze,7.20,Plain Vodka:60ml,Plain Vodka,60
1,Bay Breeze,7.20,Cranberry Juice:90ml,Cranberry Juice,90
1,Bay Breeze,7.20,Pineapple Juice:30ml,Pineapple Juice,30
2,Alabama Slammer,8.25,Southern Comfort:15ml,Southern Comfort,15
2,Alabama Slammer,8.25,Sloe Gin:15ml,Sloe Gin,15
2,Alabama Slammer,8.25,Amaretto:15ml,Amaretto,15
2,Alabama Slammer,8.25,Orange Juice:120ml,Orange Juice,120


### Calculate the price in pounds, for the required measurement of each ingredient

In [137]:
# convert the sourcing into pound first
df_sourcing = df_sourcing.merge(df_rates, on='Currency',how='left')

df_sourcing['Price'] = df_sourcing['Price'] * df_sourcing['Conversion Rate £']
df_sourcing = df_sourcing[['Ingredient','Price','ml per Bottle']]
df_sourcing

,Ingredient,Price,ml per Bottle
0,Citroen Vodka,21.9450,500
1,Chambord,26.0490,450
2,Triple Sec,16.5600,400
3,Plain Vodka,17.3736,500
4,Cranberry Juice,1.3300,1000
5,Pineapple Juice,1.8000,1000
6,Southern Comfort,28.9662,750
7,Sloe Gin,26.2086,500
8,Amaretto,18.9240,500
9,Orange Juice,1.4200,1000


In [138]:
# join the recipe list with the sourcing
df_cocktails = df_cocktails.merge(df_sourcing, left_on='Recipe',right_on='Ingredient',how='left')
df_cocktails.dtypes

Cocktail          object
Price (£)        float64
Recipe (ml)       object
Recipe            object
ml                object
Ingredient        object
Price            float64
ml per Bottle      int64
dtype: object

### Join the ingredient costs to their relative cocktails

In [139]:
# calculate the cost
df_cocktails['ml'] = df_cocktails['ml'].astype(int)
df_cocktails['cost'] = df_cocktails['ml']/df_cocktails['ml per Bottle']*df_cocktails['Price']
df_cocktails

,Cocktail,Price (£),Recipe (ml),Recipe,ml,Ingredient,Price,ml per Bottle,cost
0,Raspberry Lemon Drop,8.50,Citroen Vodka:45ml,Citroen Vodka,45,Citroen Vodka,21.9450,500,1.975050
1,Raspberry Lemon Drop,8.50,Chambord:20ml,Chambord,20,Chambord,26.0490,450,1.157733
2,Raspberry Lemon Drop,8.50,Triple Sec:20ml,Triple Sec,20,Triple Sec,16.5600,400,0.828000
3,Bay Breeze,7.20,Plain Vodka:60ml,Plain Vodka,60,Plain Vodka,17.3736,500,2.084832
4,Bay Breeze,7.20,Cranberry Juice:90ml,Cranberry Juice,90,Cranberry Juice,1.3300,1000,0.119700
5,Bay Breeze,7.20,Pineapple Juice:30ml,Pineapple Juice,30,Pineapple Juice,1.8000,1000,0.054000
6,Alabama Slammer,8.25,Southern Comfort:15ml,Southern Comfort,15,Southern Comfort,28.9662,750,0.579324
7,Alabama Slammer,8.25,Sloe Gin:15ml,Sloe Gin,15,Sloe Gin,26.2086,500,0.786258
8,Alabama Slammer,8.25,Amaretto:15ml,Amaretto,15,Amaretto,18.9240,500,0.567720
9,Alabama Slammer,8.25,Orange Juice:120ml,Orange Juice,120,Orange Juice,1.4200,1000,0.170400


### Find the total cost of each cocktail 

In [140]:
df_cocktails = df_cocktails.groupby('Cocktail').agg({'Price (£)':'mean','cost':'sum'}).reset_index()
df_cocktails

,Cocktail,Price (£),cost
0,Alabama Slammer,8.25,2.103702
1,Bay Breeze,7.20,2.258532
2,Orange Blossom,8.70,1.084484
3,Raspberry Lemon Drop,8.50,3.960783
4,Watermelon Man,7.00,4.990653


### Include a calculated field for the profit margin i.e. the difference between each cocktail's price and it's overall cost 

In [141]:
df_cocktails['margin'] = df_cocktails['Price (£)'] - df_cocktails['cost'] 
df_cocktails

,Cocktail,Price (£),cost,margin
0,Alabama Slammer,8.25,2.103702,6.146298
1,Bay Breeze,7.20,2.258532,4.941468
2,Orange Blossom,8.70,1.084484,7.615516
3,Raspberry Lemon Drop,8.50,3.960783,4.539217
4,Watermelon Man,7.00,4.990653,2.009347


### Round all numeric fields to 2 decimal places 

In [142]:
df_cocktails[['cost','margin']] = df_cocktails[['cost','margin']].round(2)
df_cocktails

,Cocktail,Price (£),cost,margin
0,Alabama Slammer,8.25,2.10,6.15
1,Bay Breeze,7.20,2.26,4.94
2,Orange Blossom,8.70,1.08,7.62
3,Raspberry Lemon Drop,8.50,3.96,4.54
4,Watermelon Man,7.00,4.99,2.01


In [143]:
df_cocktails.to_csv(r'output/2021-week11-output.csv')